# Analyse Spectrale des Réseaux de Neurones avec la Loi de Marchenko-Pastur

## Objectif

Ce notebook permet de comprendre et analyser les poids d'un réseau de neurones en utilisant la théorie des matrices aléatoires. Plus précisément, on va utiliser la **loi de Marchenko-Pastur** pour détecter les signaux importants dans les poids du réseau.

## Contexte

Quand on entraîne un réseau de neurones, les poids qu'il apprend contiennent à la fois :
- Du **signal** : des informations utiles que le réseau a appris
- Du **bruit** : des valeurs aléatoires qui ne servent pas vraiment

La loi de Marchenko-Pastur nous aide à faire la différence entre ces deux types de valeurs. On peut ensuite **élaguer** (pruning) le réseau en supprimant le bruit, ce qui le rend plus petit et plus rapide sans perdre en performance.

## Plan du Notebook

1. Définir la loi de Marchenko-Pastur
2. Vérifier qu'elle fonctionne sur une matrice aléatoire pure
3. Tester sur une matrice avec du signal ajouté
4. Entraîner un réseau de neurones sur MNIST
5. Analyser les poids du réseau
6. Élaguer le réseau en gardant seulement le signal

## 1. Imports et Configuration

On commence par importer toutes les bibliothèques nécessaires :
- **numpy** : pour les calculs numériques
- **matplotlib** : pour les graphiques
- **scipy** : pour l'algèbre linéaire (calcul des valeurs propres)
- **torch** : pour créer et entraîner le réseau de neurones
- **torchvision** : pour charger le dataset MNIST

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# On fixe les graines aléatoires pour avoir des résultats reproductibles
np.random.seed(42)
torch.manual_seed(42)

## 2. Théorie : La Loi de Marchenko-Pastur

### Qu'est-ce que c'est ?

Imaginons qu'on a une grande matrice **W** de taille N×M remplie de nombres aléatoires. La loi de Marchenko-Pastur nous dit comment se distribuent les **valeurs propres** de cette matrice.

### Formule mathématique

Pour une matrice aléatoire N×M avec :
- **c = M/N** (rapport entre nombre de colonnes et lignes)
- **σ²** (variance des éléments)

Les valeurs propres sont concentrées entre deux bornes :
- **λ_min** = σ² × (1 - √c)²
- **λ_max** = σ² × (1 + √c)²

Toute valeur propre **au-dessus de λ_max** représente du **signal réel**, pas du bruit aléatoire !

In [ ]:
def marchenko_pastur(x, sigma2, c):
    """
    Calcule la densité de la loi de Marchenko-Pastur.

    Paramètres :
    -----------
    x : array ou None
        Points où on veut calculer la densité. Si None, retourne seulement les bornes.
    sigma2 : float
        Variance des éléments de la matrice
    c : float
        Rapport M/N (colonnes/lignes)

    Retour :
    --------
    rho : densité de probabilité aux points x
    lam_min : borne inférieure de la distribution
    lam_max : borne supérieure de la distribution (seuil important !)
    """
    # Calcul des bornes théoriques
    lam_min = sigma2 * (1 - np.sqrt(c))**2
    lam_max = sigma2 * (1 + np.sqrt(c))**2

    # Si on veut juste les bornes, on s'arrête là
    if x is None:
        return None, lam_min, lam_max

    # Calcul de la densité pour chaque point x
    rho = np.zeros_like(x)
    mask = (x >= lam_min) & (x <= lam_max)  # On ne calcule que dans l'intervalle valide

    # Pour éviter les divisions par zéro
    safe_x = x.copy()
    safe_x[safe_x == 0] = 1e-16

    # Formule de la densité MP
    rho[mask] = np.sqrt((lam_max - x[mask]) * (x[mask] - lam_min)) / \
                (2 * np.pi * sigma2 * c * safe_x[mask])

    return rho, lam_min, lam_max

In [ ]:
def compute_esd(W):
    """
    Calcule la distribution spectrale empirique (ESD) d'une matrice W.

    On calcule X = (1/N) × W^T × W, puis on récupère ses valeurs propres.
    Ces valeurs propres nous disent comment l'énergie est répartie dans la matrice.

    Paramètres :
    -----------
    W : array de taille N×M
        La matrice à analyser

    Retour :
    --------
    eigenvalues : array trié
        Les valeurs propres de X, triées par ordre croissant
    """
    N, M = W.shape
    X = W.T @ W / N  # Matrice de covariance normalisée

    # eigvalsh est optimisé pour les matrices symétriques (comme X)
    return np.sort(linalg.eigvalsh(X))

## 3. Test 1 : Vérification sur une Matrice Aléatoire Pure

### Pourquoi ce test ?

Avant d'utiliser la loi MP sur un vrai réseau de neurones, on vérifie qu'elle fonctionne bien sur une matrice purement aléatoire. Si ça marche ici, on peut avoir confiance pour la suite.

### Ce qu'on fait :

1. On crée une matrice R de taille 1000×800 avec des valeurs aléatoires normales
2. On calcule ses valeurs propres (la distribution empirique)
3. On compare avec ce que prédit la théorie MP
4. On trace un graphique pour voir si ça correspond

In [ ]:
print("="*60)
print("TEST 1 : Vérification de la loi MP sur matrice aléatoire")
print("="*60)

# Création d'une matrice aléatoire
N, M = 1000, 800
R = np.random.randn(N, M) / np.sqrt(N)  # Normalisation importante !

# Calcul des valeurs propres empiriques
eigs = compute_esd(R)

# Calcul des bornes théoriques MP
_, lam_min, lam_plus = marchenko_pastur(None, 1.0 / N, M / N)

print(f"Borne inférieure théorique λ_min : {lam_min:.5e}")
print(f"Borne supérieure théorique λ+ : {lam_plus:.5e}")
print(f"Valeur propre minimale mesurée : {eigs.min():.5e}")
print(f"Valeur propre maximale mesurée : {eigs.max():.5e}")

In [ ]:
# Visualisation
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)

# Histogramme des valeurs propres mesurées
plt.hist(eigs, bins=50, density=True, alpha=0.6, label='ESD empirique')

# Courbe théorique MP
x = np.linspace(max(0, eigs.min()*0.9), eigs.max()*1.1, 300)
rho, _, _ = marchenko_pastur(x, 1.0 / N, M / N)
plt.plot(x, rho, 'r-', lw=2, label='MP théorique')

# Ligne verticale pour le seuil λ+
plt.axvline(lam_plus, color='g', ls='--', lw=2, label=f'λ+ = {lam_plus:.5e}')

plt.xlabel('λ (valeur propre)')
plt.ylabel('Densité')
plt.legend()
plt.grid(alpha=0.3)
plt.title('Matrice aléatoire R : théorie vs pratique')

print("\n✓ Si les barres bleues suivent bien la courbe rouge, la loi MP fonctionne !")

## 4. Test 2 : Modèle avec Signal (Spiked Model)

### L'idée

Maintenant on va créer une matrice qui contient **du vrai signal** en plus du bruit. On fait ça en ajoutant des "spikes" : quelques grandes valeurs singulières.

Notre matrice finale est : **W = S + R**
- **S** : matrice de signal (rang faible, quelques valeurs singulières grandes)
- **R** : matrice de bruit (aléatoire)

### Ce qu'on s'attend à voir

Les valeurs propres de W devraient :
- Suivre la loi MP dans la zone "bulk" (bruit)
- Dépasser le seuil λ+ pour les spikes (signal)

C'est exactement comme ça qu'on pourra détecter le signal dans un réseau de neurones !

In [ ]:
print("\nTEST 2 : Modèle spiked W = S + R")
print("="*60)

# Création de la matrice de signal S avec 5 spikes
rank_spikes = 5

# On crée des bases orthonormales U et V
U, _ = np.linalg.qr(np.random.randn(N, rank_spikes))
V, _ = np.linalg.qr(np.random.randn(M, rank_spikes))

# Les valeurs singulières du signal (choisies grandes pour être visibles)
spike_vals = np.array([30, 40, 50, 60, 70])
print(f"Valeurs singulières injectées : {spike_vals}")

# Construction de S = U × Diag(spike_vals) × V^T
S = U @ np.diag(spike_vals) @ V.T

# Ajout du bruit : W = S + R
W = S + R

# Analyse spectrale de W
eigs_W = compute_esd(W)
sv_W = linalg.svdvals(W)  # Valeurs singulières (racine carrée des valeurs propres)

print(f"Seuil MP (en valeurs singulières) : {np.sqrt(lam_plus * N):.4f}")
print(f"Nombre de spikes détectés au-dessus du seuil : {np.sum(sv_W > np.sqrt(lam_plus * N))}/{len(spike_vals)}")

In [ ]:
# Visualisation
plt.subplot(1, 2, 2)

# Histogramme des valeurs propres de W
plt.hist(eigs_W, bins=60, density=True, alpha=0.6, label='ESD de W')

# Théorie MP (qui prédit le bulk)
plt.plot(x, marchenko_pastur(x, 1.0 / N, M / N)[0], 'r-', lw=2, label='Bulk MP')

# Seuil λ+
plt.axvline(lam_plus, color='g', ls='--', lw=2, label=f'λ+')

plt.xlabel('λ (valeur propre)')
plt.ylabel('Densité')
plt.legend()
plt.grid(alpha=0.3)
plt.title('W = S + R (spikes visibles à droite)')
plt.tight_layout()
plt.show()

print("\n✓ Les spikes apparaissent comme des pics isolés à droite du bulk !")

## 5. Test 3 : Réseau de Neurones sur MNIST

### L'objectif

Maintenant on passe aux choses sérieuses : on va entraîner un vrai réseau de neurones sur MNIST (reconnaissance de chiffres manuscrits) et analyser ses poids avec la loi MP.

### Architecture du réseau

Un MLP (Multi-Layer Perceptron) simple :
- Entrée : 784 pixels (images 28×28)
- Couche 1 : 784 → 500 neurones + ReLU
- Couche 2 : 500 → 300 neurones + ReLU  
- Sortie : 300 → 10 classes (chiffres 0-9)

### Plan

1. Charger les données MNIST
2. Créer le réseau
3. L'entraîner pendant 5 époques
4. Analyser les poids de chaque couche

In [ ]:
print("TEST 3 : Analyse d'un réseau entraîné sur MNIST")
print("="*60)

# Définition du modèle
class MLP(nn.Module):
    """
    Réseau de neurones multi-couches pour MNIST.

    Architecture :
    - Linear 784 → 500
    - ReLU
    - Linear 500 → 300
    - ReLU
    - Linear 300 → 10
    """
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 500), nn.ReLU(),
            nn.Linear(500, 300), nn.ReLU(),
            nn.Linear(300, 10))

    def forward(self, x):
        # Aplatir l'image 28×28 en vecteur de 784 éléments
        return self.net(x.view(x.size(0), -1))

print("✓ Modèle défini")

In [ ]:
# Chargement des données MNIST
print("Chargement des données MNIST...")

# Transformations : conversion en tenseur + normalisation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Moyenne et écart-type de MNIST
])

# Téléchargement des données
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=False, transform=transform)

# DataLoaders pour itérer sur les données
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1000)

print(f"✓ Données chargées : {len(train_data)} exemples d'entraînement, {len(test_data)} de test")

In [ ]:
# Entraînement du réseau
print("\nDébut de l'entraînement...\n")

model = MLP()
criterion = nn.CrossEntropyLoss()  # Fonction de perte pour classification
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)  # Optimiseur SGD

for epoch in range(5):
    # Phase d'entraînement
    model.train()
    for imgs, labels in train_loader:
        optimizer.zero_grad()  # Réinitialiser les gradients
        loss = criterion(model(imgs), labels)  # Calculer la perte
        loss.backward()  # Rétropropagation
        optimizer.step()  # Mise à jour des poids

    # Phase d'évaluation
    model.eval()
    correct = 0
    with torch.no_grad():  # Pas besoin de calculer les gradients ici
        for imgs, labels in test_loader:
            preds = model(imgs).argmax(1)  # Prédiction = classe avec score max
            correct += (preds == labels).sum().item()

    accuracy = 100 * correct / len(test_data)
    print(f"Epoch {epoch+1}/5 - Test Accuracy: {accuracy:.2f}%")

print("\n✓ Entraînement terminé !")

## 6. Analyse Spectrale des Poids du Réseau

### Ce qu'on va faire

Pour chaque couche du réseau, on va :
1. Récupérer la matrice de poids W
2. Calculer ses valeurs singulières
3. Calculer le seuil MP pour cette couche
4. Compter combien de valeurs singulières sont au-dessus du seuil

### Interprétation

Les valeurs singulières **au-dessus** du seuil MP représentent le signal utile que le réseau a appris. Celles **en-dessous** sont du bruit qu'on peut supprimer.

In [ ]:
print("\nAnalyse spectrale des couches :")
print("="*60)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# On récupère toutes les matrices de poids (pas les biais)
weight_params = [(n, p) for n, p in model.named_parameters() if 'weight' in n]

for idx, (name, param) in enumerate(weight_params):
    # Récupération de la matrice de poids
    W = param.detach().cpu().numpy()
    Nw, Mw = W.shape

    # Calcul des valeurs singulières
    sv = linalg.svdvals(W)

    # Calcul des valeurs propres pour comparaison
    eigs = compute_esd(W)

    # Calcul du seuil MP
    _, _, lam_plus = marchenko_pastur(None, 1.0 / Nw, Mw / Nw)
    threshold = np.sqrt(lam_plus * Nw)  # Seuil en valeurs singulières

    # Visualisation
    ax = axes[idx]
    ax.plot(np.arange(len(sv)), sv, 'o', ms=3, alpha=0.6, label='Valeurs singulières')
    ax.axhline(threshold, color='r', ls='--', lw=2, label=f'Seuil MP={threshold:.3f}')
    ax.set_yscale('log')  # Échelle logarithmique pour mieux voir
    ax.set_xlabel('Indice')
    ax.set_ylabel('Valeur singulière')
    ax.set_title(f'{name} ({Nw}×{Mw})')
    ax.legend()
    ax.grid(alpha=0.3)

    # Comptage des valeurs au-dessus du seuil
    n_signal = int(np.sum(sv > threshold))
    total = min(Nw, Mw)
    percentage = 100 * n_signal / total

    print(f"{name:20s}: {n_signal:3d}/{total} val. sing. > seuil ({percentage:.1f}%)")

plt.tight_layout()
plt.show()

print("\n✓ Les points au-dessus de la ligne rouge sont le signal, ceux en-dessous sont du bruit")

## 7. Pruning (Élagage) Basé sur la Loi MP

### L'idée

Maintenant qu'on sait quelles valeurs singulières sont du bruit, on va créer une version élaguée du réseau en :
1. Faisant une SVD (décomposition en valeurs singulières) de chaque matrice de poids
2. Mettant à zéro toutes les valeurs singulières en-dessous du seuil MP
3. Reconstruisant la matrice avec seulement le signal

### Avantages

- Réseau plus petit (moins de paramètres effectifs)
- Plus rapide à l'inférence
- Peut généraliser mieux (moins d'overfitting)
- Perte de performance minimale si bien fait

In [ ]:
print("\nPruning MP-based :")
print("="*60)

# Création d'une copie du modèle
pruned_model = MLP()
pruned_model.load_state_dict(model.state_dict())

for name, param in pruned_model.named_parameters():
    if 'weight' in name:
        # Récupération de la matrice
        W = param.detach().cpu().numpy()
        Nw, Mw = W.shape

        # Calcul du seuil MP
        _, _, lam_plus = marchenko_pastur(None, 1.0 / Nw, Mw / Nw)
        threshold = np.sqrt(lam_plus * Nw)

        # SVD : W = U × S × V^T
        U, s, Vt = linalg.svd(W, full_matrices=False)

        # Mise à zéro des valeurs singulières < seuil
        s_pruned = s.copy()
        s_pruned[s_pruned < threshold] = 0.0

        # Reconstruction de W
        W_approx = (U * s_pruned) @ Vt  # Broadcasting efficace

        # Mise à jour des poids du modèle
        with torch.no_grad():
            param.data.copy_(torch.from_numpy(W_approx).to(param.data.dtype))

        n_kept = int(np.sum(s_pruned > 0))
        total = min(Nw, Mw)
        print(f"{name:20s}: {n_kept:3d}/{total} val. sing. gardées")

print("\n✓ Pruning terminé !")

## 8. Évaluation : Comparaison des Performances

### Question importante

Est-ce que notre élagage a nui aux performances du réseau ?

On va comparer l'accuracy (précision) du modèle original et du modèle élagué sur le jeu de test.

In [ ]:
# Fonction d'évaluation
def evaluate(m):
    """
    Calcule la précision d'un modèle sur le jeu de test MNIST.

    Paramètres :
    -----------
    m : nn.Module
        Le modèle à évaluer

    Retour :
    --------
    accuracy : float
        Précision en pourcentage (0-100)
    """
    correct = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            preds = m(imgs).argmax(1)
            correct += (preds == labels).sum().item()
    return 100 * correct / len(test_data)

# Évaluation des deux modèles
model.eval()
pruned_model.eval()

acc_orig = evaluate(model)
acc_pruned = evaluate(pruned_model)

print("\n" + "="*60)
print("RÉSULTATS FINAUX")
print("="*60)
print(f"Accuracy originale       : {acc_orig:.2f}%")
print(f"Accuracy après pruning   : {acc_pruned:.2f}%")
print(f"Différence               : {acc_pruned - acc_orig:+.2f}%")
print("="*60)

if acc_pruned >= acc_orig - 1.0:
    print("\n✓ SUCCÈS : Le pruning MP préserve les performances !")
else:
    print("\n⚠ ATTENTION : Le pruning a causé une baisse de performance significative.")

## Conclusion

### Ce qu'on a appris

1. **La loi de Marchenko-Pastur** décrit comment se distribuent les valeurs propres d'une matrice aléatoire

2. Dans un réseau de neurones entraîné, les poids contiennent :
   - Du **signal** (valeurs singulières au-dessus du seuil MP)
   - Du **bruit** (valeurs singulières en-dessous du seuil)

3. On peut **élaguer** le réseau en supprimant le bruit sans perdre beaucoup de performance

### Applications pratiques

- Compression de modèles pour mobile/edge devices
- Accélération de l'inférence
- Meilleure généralisation (réduction de l'overfitting)
- Compréhension de ce que le réseau a vraiment appris

### Pour aller plus loin

- Tester sur d'autres architectures (CNN, Transformers)
- Comparer avec d'autres méthodes de pruning (magnitude, gradient-based)
- Fine-tuning après pruning pour récupérer de la performance
- Analyse de la stabilité de la solution